In [238]:
from data_encode import encode
from data_encode import to_byte

In [239]:
# string_input = input("Enter the data to be encoded(14 chars max)")
string_input = "HelloWorld"
encoded_message = encode(string_input)

print(encoded_message)

[64, 164, 134, 86, 198, 198, 245, 118, 247, 38, 198, 64, 236, 17, 236, 17]


In [240]:
alpha = 0x02 #Alpha is an an element which can generate all elements of the field after being raiesd to some power
primitive = 0x11D #Primitive is irreductible polynomial that defines the Gallois Field.

log = [0]*256
antilog = [0]*512 #Since the operations are closed so there must be wraparound i.e. alpha^255 = alpha^0 = 1 so when there is antilog(log[a] + log[b]), antilog(log[a] + log[b] % 255) is not required since the values beyond 255 are kept the same as the initial values.

x = 1

for i in range(255):
    antilog[i] = x
    log[x] = i

    x = x<<1 #Left Shift by 1 i.e. multiply by 2 i.e. alpha; microprocessor nostalgia
    
    if x & 0x100: #If there is 9th bit then it is beyond GF(256)
        x = x^primitive #EXOR is same as addition/substraction, later in this case, in GF(2)

for i in range(255, 512):
    antilog[i] = antilog[i-255]

In [241]:
def gf_addition(a, b):
    return a^b

def gf_multiplication(a, b):
    if a==0 or b==0:
        return 0
    return antilog[log[a] + log[b]]

def gf_division(a, b):
    if a==0:
        return 0
    if b==0:
        raise ZeroDivisionError()
    return antilog[(log[a] - log[b])%255]

def poly_mul(a, b):
    result = [0]*(len(a)+len(b)-1)
    for i in range(len(a)):
        for j in range(len(b)):
            result[i+j] ^= gf_multiplication(a[i], b[j])
    return result

def generate_generator(num_ec):
    g = [1]
    for i in range(num_ec):
        g = poly_mul(g, [1, antilog[i]])  # (x-a^i) is written as [1, antilog[i] i.e. a^i] and +/- are same in GF(256)
    return g

In [242]:
def poly_div(dividend, divisor): #We perform polynomial long division after which the remainder will be the error correction codeword
    divi = list(dividend)
    for i in range(len(dividend) - len(divisor)+1):
        coeff = divi[i]
        if coeff!=0:
            for j in range(len(divisor)):
                divi[i+j] ^= gf_multiplication(divisor[j], coeff) 
                '''This is in place division where the dividend is modified in each operation 
                    divisor*highest_coeff_of_dividend is substracted from dividend taking the first terms equal to the number of terms of divisor.
                    The first term is always cancelled as the generator polynomial is monic and other terms are modified accordingly.'''
    remainder = divi[-(len(divisor)-1):]  #The last remaining terms with degree, at most, one less than divisor
    return remainder

In [243]:
#Error Correction level: 
L = 0b01
M = 0b00
Q = 0b11
H = 0b10
ERROR_CORRECTION_BYTES = 10

In [244]:
padded_message = encoded_message + [0 for i in range(ERROR_CORRECTION_BYTES)]
print(padded_message)

[64, 164, 134, 86, 198, 198, 245, 118, 247, 38, 198, 64, 236, 17, 236, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [245]:
redundant_bits = poly_div(padded_message, generate_generator(ERROR_CORRECTION_BYTES))
print(redundant_bits)

[205, 22, 167, 68, 166, 106, 114, 197, 70, 229]


In [246]:
for i, j in zip(range(len(encoded_message), len(encoded_message) + ERROR_CORRECTION_BYTES), range(ERROR_CORRECTION_BYTES)):
    padded_message[i] = redundant_bits[j]

final_codeword = padded_message
print(final_codeword)

[64, 164, 134, 86, 198, 198, 245, 118, 247, 38, 198, 64, 236, 17, 236, 17, 205, 22, 167, 68, 166, 106, 114, 197, 70, 229]


In [247]:
import numpy as np

In [248]:
# To build the 21*21 matrix with required patterns

code_matrix = np.zeros((21, 21), dtype = int)
is_function = np.zeros((21, 21), dtype = bool)
SIZE = 21

In [249]:
#Finder 7*7 matrix with alternate black and white patches at top left, top right and bottom left

finder = np.array([
    [0, 0, 0, 0, 0, 0, 0],  
    [0, 1, 1, 1, 1, 1, 0],  
    [0, 1, 0, 0, 0, 1, 0],  
    [0, 1, 0, 0, 0, 1, 0],  
    [0, 1, 0, 0, 0, 1, 0],  
    [0, 1, 1, 1, 1, 1, 0],  
    [0, 0, 0, 0, 0, 0, 0],  
])
def place_finder_separator(row, column):
    for i in range(7):
        for j in range(7):
            code_matrix[row+i, column+j] = finder[i, j]
            is_function[row+i, column+j] = True

    for i in range(-1, 8):
        rr = row+i
        if (rr>=0 and rr<SIZE):
            for j in range(-1, 8):
                cc = column + j
                if (cc>=0 and cc<SIZE and not is_function[rr, cc]):
                    code_matrix[rr, cc] = 1
                    is_function[rr, cc] = True
place_finder_separator(0, 0)
place_finder_separator(SIZE - 7, 0)
place_finder_separator(0, SIZE - 7)



In [250]:
#To place timing
def place_timing():
    init = 0
    for i in range(8, (SIZE - 8)):
        code_matrix[i, 6] = init
        init = abs(1 - init)
        is_function[i, 6] = True
    init = 0
    for j in range(8, SIZE - 8):
        code_matrix[6, j] = init
        init = abs(1 - init)
        is_function[6, j] = True
place_timing()
code_matrix[13, 8] = 0
is_function[13, 8] = True



In [251]:
#Reserve format strip area
for i in range(9):
    if not is_function[i, 8]:
        is_function[i, 8] = True  
        
for j in range(8):
    if not is_function[8, j]:
        is_function[8, j] = True

for i in range(7):
    is_function[i+(SIZE - 7), 8] = True
for j in range(8):
    is_function[8, j+(SIZE - 8)] = True



In [252]:
#Now time for data placement
data = []

for x in final_codeword:
    for y in to_byte(x, 8):
        data.append(int(y))

direction = -1
column = SIZE - 1
data_index = 0
placed = 0

while column>0:
    if column==6:
        column = column - 1
    row = (SIZE - 1) if direction == -1 else 0

    while row >=0 and row<SIZE:
        for col in [column, column - 1]:
            if not is_function[row, col]:
                if data_index>=len(data):
                    break
                code_matrix[row, col] = data[data_index]
                data_index += 1
                placed+=1
        row += direction
        
    column -= 2
    direction *= -1


In [253]:
#To apply masking
'''
0. (i+j)%2==0 (alternates black and white pattern)
1. i%2==0  
2. j%3==0
3. (i+j)%3==0
4. (floor(i/2) + floor(j/3))%2==0
5. ((i*j)%2) + ((i*j)%3)==0
6. (((i*j)%2) + ((i*j)%3))%2==0
7. (((i+j)%2)+((i*j)%3))%2==0
'''


def apply_mask(code_matrix):
    mask0 = code_matrix.copy()
    mask1 = code_matrix.copy()
    mask2 = code_matrix.copy()
    mask3 = code_matrix.copy()
    mask4 = code_matrix.copy()
    mask5 = code_matrix.copy()
    mask6 = code_matrix.copy()
    mask7 = code_matrix.copy()
    
    for i in range(SIZE):
        for j in range(SIZE):
            if (i+j)%2 == 0 and not is_function[i, j]:
                mask0[i, j] = abs(mask0[i, j] - 1)
    for i in range(SIZE):
        for j in range(SIZE):
            if i%2 == 0 and not is_function[i, j]:
                mask1[i, j] = abs(mask1[i, j] - 1)
    for i in range(SIZE):
        for j in range(SIZE):
            if j%3 == 0 and not is_function[i, j]:
                mask2[i, j] = abs(mask2[i, j] - 1)
    for i in range(SIZE):
        for j in range(SIZE):
            if (i+j)%3 == 0 and not is_function[i, j]:
                mask3[i, j] = abs(mask3[i, j] - 1)
    for i in range(SIZE):
        for j in range(SIZE):
            if ((i//2) + (j//3))%2==0 and not is_function[i, j]:
                mask4[i, j] = abs(mask4[i, j] - 1)
    for i in range(SIZE):
        for j in range(SIZE):
            if ((i*j)%2) + ((i*j)%3)==0 and not is_function[i, j]:
                mask5[i, j] = abs(mask5[i, j] - 1)
    for i in range(SIZE):
        for j in range(SIZE):
            if (((i*j)%2) + ((i*j)%3))%2==0 and not is_function[i, j]:
                mask6[i, j] = abs(mask6[i, j] - 1)
    for i in range(SIZE):
        for j in range(SIZE):
            if (((i+j)%2)+((i*j)%3))%2==0 and not is_function[i, j]:
                mask7[i, j] = abs(mask7[i, j] - 1)

    return mask0, mask1, mask2, mask3, mask4, mask5, mask6, mask7

In [254]:
mask0, mask1, mask2, mask3, mask4, mask5, mask6, mask7 = apply_mask(code_matrix)

In [255]:
#To determine the best mask

'''
-Penalty for each group of five or more same colored modules in a row or column: +3 for run of 5, +1 for each extra module,
-Penalty for each 2*2 area of same colored modules: +3 for each 2*2 area,
-Penalty if some patterns are similar to finder patterns: +40 for each finder-like pattern,
-Penalty for unbalance ration of black and white: calculate % of dark modules, for 5% away form 50%, penalty of 10 is given.
'''

def evaluate_penalty(mask_matrix):
    penalty = 0
    #Five or more same colored modules in a row or column
    row_run = 0
    column_run = 0
    previous_bit_row = -1
    previous_bit_column = -1
    for i in range(SIZE):
        for j in range(SIZE):
            if mask_matrix[i, j] == previous_bit_row:
                row_run+=1
            else:
                if row_run >=5:
                    penalty += 3 + (row_run - 5)
                row_run = 1
                
            previous_bit_row = mask_matrix[i, j]

            if mask_matrix[j, i] == previous_bit_column:
                column_run += 1
            else: 
                if column_run>=5:
                    penalty += 3 + (column_run - 5)
                column_run = 1
            previous_bit_column = mask_matrix[j, i]
        if row_run >= 5:
            penalty += 3 + (row_run - 5)
        if column_run >= 5:
            penalty += 3 + (column_run - 5)
        
    #2*2 area of same colored modules
    for i in range(SIZE - 1):
        for j in range(SIZE - 1):
            if mask_matrix[i, j] == mask_matrix[i+1, j] == mask_matrix[i, j+1] == mask_matrix[i+1, j+1]:
                penalty += 3


    #Finder like patterns
    core_finder_pattern = np.array([0, 1, 0, 0, 0, 1, 0])
    for row in mask_matrix:
        for i in range(SIZE - 6):
            if np.array_equal(row[i:i+7], core_finder_pattern):
                penalty+=40

    for j in range(SIZE):
        for i in range(SIZE - 6):
            if np.array_equal(mask_matrix[i:i+7, j], core_finder_pattern):
                penalty+=40


    #Proportion of Black and White units
    num_white = np.sum(mask_matrix)
    total = SIZE*SIZE
    white_percent = ((total - num_white) / total)*100
    away = abs(white_percent - 50)
    if away>=5:
        penalty+= int((away//5))*10
    
    return penalty
    

In [256]:
mask_penalties = [evaluate_penalty(mask0), evaluate_penalty(mask1), evaluate_penalty(mask2), evaluate_penalty(mask3), evaluate_penalty(mask4), evaluate_penalty(mask5), evaluate_penalty(mask6), evaluate_penalty(mask7)]
best_mask = np.argmin(mask_penalties)
print(best_mask)

4


In [257]:
#Format String
''' 
Error correction level: 
 -L = 01
 -M = 00
 -Q = 11
 -H = 10

 Masking pattern(3 - bit)

 The rest is BCH error correction(10 bit) for the above 5 bits
'''
ec_level = '00'
mask_number = to_byte(best_mask, 3)

format_string = ec_level + mask_number
format_string += '0'*10

#Generator polynomial is x^10 + x^8 + x^5 + x^4 + x^2 + x + 1
generator_bch = '10100110111'
generator_bch = [int(generator_bch[i]) for i in range(len(generator_bch)) ]

format_string = [int(format_string[i]) for i in range(len(format_string))]

def bch_poly_div(dividend, divisor):
    divid = list(dividend)
    for i in range(len(dividend) - len(divisor) + 1):
        coeff = divid[i]
        if coeff !=0:
            for j in range(len(divisor)):
                divid[i+j] ^= divisor[j]
    remainder = divid[-(len(divisor) - 1):]
    return remainder

format_redundancy = bch_poly_div(format_string, generator_bch)

format_string[5:] = format_redundancy

# A format string mask is applied to the format string
format_mask = '101010000010010'
format_mask = [int(format_mask[i]) for i in range(len(format_mask))]

format_string = [format_mask[i]^format_string[i] for i in range(len(format_string))]

format_index = 0
for j in range(8):
    if j!=6:
        code_matrix[8, j] = format_string[format_index]
        format_index += 1

for i in reversed(range(9)):
    if i!=6:
        code_matrix[i, 8] = format_string[format_index]
        format_index += 1 
        
format_index = 0
for j in range(8):
    code_matrix[8, j+(SIZE - 8)] = format_string[format_index]
    format_index += 1


for i in reversed(range(7)):
    code_matrix[i+(SIZE - 7), 8] = format_string[format_index]
    format_index+=1



In [258]:
from PIL import Image

In [259]:
SCALE = 10

img_array = np.uint8(code_matrix*255)
img = Image.fromarray(img_array, mode = 'L')
img = img.resize((SIZE*SCALE, SIZE*SCALE), Image.NEAREST)
img.show()